In [1]:
import pandas as pd
import numpy as np
import schedule
import os
import math
import datetime
from datetime import timedelta
from datetime import datetime
import matplotlib.pyplot as plt
import json
from polygeohasher import polygeohasher
from shapely.geometry import Point, Polygon
import plotly.express as px
import datetime as dt

In [2]:
allSanFran = pd.read_csv('./cabs.csv')

In [3]:
#come prima cosa ho da modificare il formato di timestamp e convertirlo in datetime
allSanFran['timestamp'] = pd.to_datetime(allSanFran['timestamp'], unit='s')

allSanFran

#primo problema, ho il dataset al contrario!
#per prima cosa troviamo l'area di appartenza con geohash

,driver,lat,lon,occupied,timestamp
0,egteir,37.75135,-122.39526,0,2008-06-04 08:25:54
1,egteir,37.75143,-122.39529,0,2008-06-04 08:24:08
2,egteir,37.75154,-122.39531,0,2008-06-04 08:20:24
3,egteir,37.75154,-122.39531,0,2008-06-04 08:19:37
4,egteir,37.75159,-122.39530,0,2008-06-04 08:18:29
...,...,...,...,...,...
11219950,ashgati,37.75128,-122.39483,0,2008-05-17 13:17:48
11219951,ashgati,37.75127,-122.39441,0,2008-05-17 13:16:11
11219952,ashgati,37.75154,-122.39453,0,2008-05-17 13:15:06
11219953,ashgati,37.75157,-122.39452,0,2008-05-17 13:14:05


In [4]:
import pygeohash as gh

allSanFran['geohash_area']=allSanFran.apply(lambda x: gh.encode(x.lat, x.lon, precision=6), axis=1)


In [25]:
#parte di visualizzazione
import geopandas as gpd

gdf = gpd.read_file('./prova.geojson')
primary_df = polygeohasher.create_geohash_list(gdf,6,inner=False)
geo_df = polygeohasher.geohashes_to_geometry(primary_df,"geohash_list")
geo_df.to_file("SanFranciscoGrid.geojson",driver = "GeoJSON")
SanFranGrid = json.load(open('SanFranciscoGrid.geojson','r'))


In [26]:
import folium
m = folium.Map([37.80, -112.26], zoom_start=12, tiles='cartodbpositron')
folium.GeoJson(SanFranGrid, name="geojson").add_to(m)
m

In [7]:
allSanFran

,driver,lat,lon,occupied,timestamp,geohash_area
0,egteir,37.75135,-122.39526,0,2008-06-04 08:25:54,9q8yyb
1,egteir,37.75143,-122.39529,0,2008-06-04 08:24:08,9q8yyb
2,egteir,37.75154,-122.39531,0,2008-06-04 08:20:24,9q8yyb
3,egteir,37.75154,-122.39531,0,2008-06-04 08:19:37,9q8yyb
4,egteir,37.75159,-122.39530,0,2008-06-04 08:18:29,9q8yyb
...,...,...,...,...,...,...
11219950,ashgati,37.75128,-122.39483,0,2008-05-17 13:17:48,9q8yyb
11219951,ashgati,37.75127,-122.39441,0,2008-05-17 13:16:11,9q8yyb
11219952,ashgati,37.75154,-122.39453,0,2008-05-17 13:15:06,9q8yyb
11219953,ashgati,37.75157,-122.39452,0,2008-05-17 13:14:05,9q8yyb


In [8]:
#Molte meno celle rispetto a Pechino, chiedere se per caso sia meglio utilizzare geohashing 6 caratteri invece di 5
#potrebbe essere fatto in poco tempo, nel frattempo inizio con la velocità e la distanza.
#altro problema è che il dataset è al contrario molto probabilmente.





Da quello che ho capito aprendo il dataset, sono registrati tot. guidatori per i giorni che vanno dal 17-05 al 06-04, con un ordine cronologico che va dal più recente al meno recente (Il contrario di quello di Pechino).

Possbili idea: scorrere il dataset al contraio -> la più intelligente, non dovrei avere grossi problemi.


In [9]:
latlonSanFran=allSanFran[['driver','lon','lat']].to_numpy() #copio la lista di coordinate e di driver in questo array

In [10]:
#iniziamo a calcolare la distanza tra ogni punto

#Funzione che mi implementa la forumla di Haversine per calcolare la distanza che c'è tra due punti successivi
from math import radians, cos, sin, asin, sqrt
def measure(lat1, lon1, lat2, lon2) :
    
    R = 6372.8 

    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)

    a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
    c = 2*asin(sqrt(a))

    return R * c

In [11]:
#Calcolo la distanza che c'è tra ogni punto al fine di potermi calcolare la velocità

i = len(latlonSanFran)-1
lista = [] 
id= latlonSanFran[i][0]
while(i>=0):
    if(id!=latlonSanFran[i-1][0]):
        
        lista.append('end')
        id=latlonSanFran[i-1][0]
        
    else:
        lista.append(measure(latlonSanFran[i-1][2],latlonSanFran[i-1][1],latlonSanFran[i][2],latlonSanFran[i][1])*1000)
    i=i-1

In [12]:
lista1 = lista.reverse() #mi permette di rovesciare la lista delle distanze

In [13]:
allSanFran['distances_m'] = lista

In [14]:
allSanFran

,driver,lat,lon,occupied,timestamp,geohash_area,distances_m
0,egteir,37.75135,-122.39526,0,2008-06-04 08:25:54,9q8yyb,end
1,egteir,37.75143,-122.39529,0,2008-06-04 08:24:08,9q8yyb,9.281003
2,egteir,37.75154,-122.39531,0,2008-06-04 08:20:24,9q8yyb,12.360678
3,egteir,37.75154,-122.39531,0,2008-06-04 08:19:37,9q8yyb,0.0
4,egteir,37.75159,-122.39530,0,2008-06-04 08:18:29,9q8yyb,5.630422
...,...,...,...,...,...,...,...
11219950,ashgati,37.75128,-122.39483,0,2008-05-17 13:17:48,9q8yyb,98.898318
11219951,ashgati,37.75127,-122.39441,0,2008-05-17 13:16:11,9q8yyb,36.95322
11219952,ashgati,37.75154,-122.39453,0,2008-05-17 13:15:06,9q8yyb,31.831416
11219953,ashgati,37.75157,-122.39452,0,2008-05-17 13:14:05,9q8yyb,3.450736


Trovata la distanza tra ogni punto, adesso mi serve trovare gli intervalli di tempo tra ogni coppia di record

In [15]:
#funzione che permette di trovare la differenza tra due datetime
def timeSet(datetime1,datetime2):
    fmt = '%Y-%d-%m %H:%M:%S'
    tstamp1 = datetime.strptime(datetime1, fmt)
    tstamp2 = datetime.strptime(datetime2, fmt)

    if tstamp1 > tstamp2:
        td = tstamp1 - tstamp2
    else:
        td = tstamp2 - tstamp1
    return td.total_seconds()

In [16]:
record = allSanFran[['driver','timestamp']].to_numpy()

In [17]:
#funzione che mi permette di trovare gli intervalli di registrazione tra ogni coppia di record
def ctime(latlonSanFran):
    result = []
    intervaltime=len(latlonSanFran)-1
    i = intervaltime
    id=latlonSanFran[i][0]
    while (i>=0):
        if(id!=latlonSanFran[i-1][0]):
            result.append('end')
            id=latlonSanFran[i-1][0]
        elif(timeSet(latlonSanFran[i-1][1].strftime("%Y-%d-%m %H:%M:%S"),latlonSanFran[i][1].strftime("%Y-%d-%m %H:%M:%S")) > 3600.0):
            result.append("sosta")
        else:
            result.append(timeSet(latlonSanFran[i-1][1].strftime("%Y-%d-%m %H:%M:%S"),latlonSanFran[i][1].strftime("%Y-%d-%m %H:%M:%S")))
        i=i-1;
    return result

In [18]:
resultRecordTime = ctime(record)

In [19]:
resultRecordTime1 = resultRecordTime.reverse()

In [20]:
allSanFran['IntervalRecordTime_s'] = resultRecordTime

In [21]:
allSanFran

,driver,lat,lon,occupied,timestamp,geohash_area,distances_m,IntervalRecordTime_s
0,egteir,37.75135,-122.39526,0,2008-06-04 08:25:54,9q8yyb,end,end
1,egteir,37.75143,-122.39529,0,2008-06-04 08:24:08,9q8yyb,9.281003,106.0
2,egteir,37.75154,-122.39531,0,2008-06-04 08:20:24,9q8yyb,12.360678,224.0
3,egteir,37.75154,-122.39531,0,2008-06-04 08:19:37,9q8yyb,0.0,47.0
4,egteir,37.75159,-122.39530,0,2008-06-04 08:18:29,9q8yyb,5.630422,68.0
...,...,...,...,...,...,...,...,...
11219950,ashgati,37.75128,-122.39483,0,2008-05-17 13:17:48,9q8yyb,98.898318,61.0
11219951,ashgati,37.75127,-122.39441,0,2008-05-17 13:16:11,9q8yyb,36.95322,97.0
11219952,ashgati,37.75154,-122.39453,0,2008-05-17 13:15:06,9q8yyb,31.831416,65.0
11219953,ashgati,37.75157,-122.39452,0,2008-05-17 13:14:05,9q8yyb,3.450736,61.0


Trovato l'intervallo di tempo, adesso posso calcolare la velocità tra ogni punto, successivamente l'accelerazione

In [22]:
#funzione che mi permette di calcolare la velocità tra ogni record 
def measure_speed(lista,resultRecordTime):
    intervalSpeed=len(resultRecordTime)-1 #lista and timex have same lenght
    speedList= []
    i=intervalSpeed
    while (i>=0):
        if(resultRecordTime[i] == "sosta" or resultRecordTime[i]=="end" or resultRecordTime[i]=="end" or resultRecordTime[i]==0):
            speedList.append(0.0)
        elif((lista[i]/(resultRecordTime[i]))*3.6 > 150.0):
            speedList.append("assurdo")
        else:
            speedList.append((lista[i]/(resultRecordTime[i]))*3.6) #in k/h
        i=i-1    
    return speedList

In [23]:
resultSpeed = measure_speed(lista,resultRecordTime)

In [24]:
resultSpeed1 = resultSpeed.reverse()

In [25]:
allSanFran['speed_km'] = resultSpeed

In [26]:
allSanFran

,driver,lat,lon,occupied,timestamp,geohash_area,distances_m,IntervalRecordTime_s,speed_km
0,egteir,37.75135,-122.39526,0,2008-06-04 08:25:54,9q8yyb,end,end,0.0
1,egteir,37.75143,-122.39529,0,2008-06-04 08:24:08,9q8yyb,9.281003,106.0,0.315204
2,egteir,37.75154,-122.39531,0,2008-06-04 08:20:24,9q8yyb,12.360678,224.0,0.198654
3,egteir,37.75154,-122.39531,0,2008-06-04 08:19:37,9q8yyb,0.0,47.0,0.0
4,egteir,37.75159,-122.39530,0,2008-06-04 08:18:29,9q8yyb,5.630422,68.0,0.298081
...,...,...,...,...,...,...,...,...,...
11219950,ashgati,37.75128,-122.39483,0,2008-05-17 13:17:48,9q8yyb,98.898318,61.0,5.836622
11219951,ashgati,37.75127,-122.39441,0,2008-05-17 13:16:11,9q8yyb,36.95322,97.0,1.37146
11219952,ashgati,37.75154,-122.39453,0,2008-05-17 13:15:06,9q8yyb,31.831416,65.0,1.762971
11219953,ashgati,37.75157,-122.39452,0,2008-05-17 13:14:05,9q8yyb,3.450736,61.0,0.20365


Devo trovare gli starting point e gli ending point di ogni veicolo

Quasi tutti i veicoli sono registrati tutto il giorno per tutti i giorni, magari prendo ending point che hanno una sosta maggiore di 1h

In [27]:
def start(allSanFran):
    i = len(allSanFran)-1
    lon = [allSanFran['lon'][i]]
    lat = [allSanFran['lat'][i]]
    driver = [allSanFran['driver'][i]]
    
    while(i>=0):
        if((allSanFran['IntervalRecordTime_s'][i] == 'sosta' or allSanFran['distances_m'][i]== 'end')and i !=0):
            lon.append(allSanFran['lon'][i-1])
            lat.append(allSanFran['lat'][i-1])
            driver.append(allSanFran['driver'][i-1])
        i = i-1
    list_of_tuples = list(zip(driver,lon,lat))
    list_of_tuples
    startDf = pd.DataFrame(list_of_tuples,columns = ['driver','longitude','latitude'])
    return startDf

def end(allSanFran):
    i = len(allSanFran)-1
    lon = []
    lat = []
    driver = []
    
    while(i>=0):
        if(allSanFran['IntervalRecordTime_s'][i] == 'sosta' or allSanFran['distances_m'][i]== 'end'):
            lon.append(allSanFran['lon'][i])
            lat.append(allSanFran['lat'][i])
            driver.append(allSanFran['driver'][i])
        i = i-1
    list_of_tuples = list(zip(driver,lon,lat))
    list_of_tuples
    endDf = pd.DataFrame(list_of_tuples,columns = ['driver','longitude','latitude'])
    return endDf

In [28]:
allStart = start(allSanFran)
allEnd = end(allSanFran)

In [29]:
allStart

,driver,longitude,latitude
0,ashgati,-122.39448,37.75155
1,ashgati,-122.40603,37.77532
2,ashgati,-122.39364,37.75149
3,ashgati,-122.39404,37.75056
4,ashgati,-122.39422,37.75218
...,...,...,...
18302,egteir,-122.38587,37.61657
18303,egteir,-122.38827,37.61609
18304,egteir,-122.39533,37.75035
18305,egteir,-122.40296,37.74823


In [30]:
allEnd

,driver,longitude,latitude
0,ashgati,-122.39527,37.75158
1,ashgati,-122.39528,37.75144
2,ashgati,-122.39388,37.75062
3,ashgati,-122.39418,37.75193
4,ashgati,-122.39432,37.75192
...,...,...,...
18302,egteir,-122.39278,37.61445
18303,egteir,-122.41899,37.76991
18304,egteir,-122.39515,37.75116
18305,egteir,-122.39264,37.75134


In [31]:
#geohashing start e end dtaframes

allStart['geohash_area']=allStart.apply(lambda x: gh.encode(x.latitude, x.longitude, precision=6), axis=1)
allEnd['geohash_area']=allEnd.apply(lambda x: gh.encode(x.latitude, x.longitude, precision=6), axis=1)


In [33]:
#creo gli slot temporali per verificare quali sono le ore con maggior numero di veicoli
allSanFran['timestamp']=pd.to_datetime(allSanFran['timestamp'],format='%Y-%d-%m %H:%M:%S')
allSanFran=allSanFran.assign(session=pd.cut(allSanFran.timestamp.dt.hour,
                            [0,6,12,18,23],
                            labels=['Night','Morning','Afternoon','Evening'],
                            include_lowest=True))

In [34]:
allSanFran.to_csv('./allSanFran.csv')

Posso iniziare adesso a considerare le aree

In [27]:
from shapely import wkt
geo_df.to_csv("Tassellation.csv") #solito problema per ordinare il geodataframe
tasselation = pd.read_csv("./Tassellation.csv")
del tasselation["Unnamed: 0"]
tasselation['geometry'] = tasselation['geometry'].apply(wkt.loads)
geo_tassellation = gpd.GeoDataFrame(
    tasselation,geometry='geometry')
#risolto problema

In [65]:
allArea = pd.DataFrame(geo_tassellation['geohash_list']) #Dataframe della griglia

In [36]:
allSanFran = allSanFran.drop(allSanFran[allSanFran.speed_km == 'assurdo'].index) #alcune coordinate sono errate

In [37]:
#prima copia nel caso in cui avessi problemi in futuro
allSanFran.to_csv('./all1.csv')
allStart.to_csv('allStart.csv')
allEnd.to_csv('allEnd.csv')

In [114]:
#Funzione che conta i numero di punti per area
def Count1(allArea,allS):
    i = 0
    result = []
    areaLen = len(allArea['geohash_list'])
    while(i<areaLen):
        areaStart = allS[allS.geohash_area == allArea['geohash_list'][i]]
        result.append(len(areaStart['geohash_area']))
        i=i+1
    return result

In [115]:
resultCountStart = Count1(allArea,allStart) #conto numero veicoli in partenza da un'area
resultCountEnd = Count1(allArea,allEnd) #conto numero di veicoli in arrivo in un area
resultCountTot = Count1(allArea,allSanFran) #conto numero di veicoli che transitano in un area per spazio temporale pari a 3 settimane

In [116]:
allArea['StartingVehicles']= resultCountStart
allArea['EndingVehicles'] = resultCountEnd
allArea['TotNumberRecordVehicles'] = resultCountTot

In [117]:
#funzione che trova la velocità massima di una determinata area i
def SpeedMax(allSanFran,allArea):
    allAreaLen= len(allArea['geohash_list'])
    i=0
    result = []
    while(i<allAreaLen):
        area= allSanFran[allSanFran.geohash_area == allArea['geohash_list'][i]]
        speed = area['speed_km']
        result.append(speed.max())
        i=i+1
    return result

In [118]:
resultSpeedMax = SpeedMax(allSanFran,allArea)

In [119]:
allArea['maxSpeed_km/h'] = resultSpeedMax #aggiungo l'attributo velocità al dataframe

In [120]:
#funzione che mi ritorna la velocità media di una determinata area
def AverageSpeed(allSanFran,allArea):
    allAreaLen= len(allArea['geohash_list'])
    i=0
    result = []
    while(i<allAreaLen):
        area= allSanFran[allSanFran.geohash_area == allArea['geohash_list'][i]]
        speed = area['speed_km']
        result.append(speed.mean())
        i=i+1
    return result

In [121]:
resultAverageSpeed= AverageSpeed(allSanFran,allArea)

In [122]:
allArea['AverageSpeed'] = resultAverageSpeed #aggiungo l'attributo velocità media al dataframe

In [123]:
allStart['end'] = allEnd['geohash_area'] #aggiorno il dataframe di inzio con l'attributo end
allEnd['start'] = allStart['geohash_area'] #aggiorno il dataframe di fine con l'attributo start

In [124]:
#cerco di vedere quanti sono i veicoli entranti ed i veicoli uscenti in una determinata area 
#uscenti -> iniziano in un area specificat e terminano la corsa in un'altra area
#entranti -> iniziano la corsa in una certa area e terminano la corsa in una specifica area


def checkOutEdge(allArea,allStart):
    i = 0
    count = 0
    result = []
    areaLen = len(allArea['geohash_list'])
    while(i<areaLen):
        j=0
        count=0
        areaStart = allStart[allStart.geohash_area == allArea['geohash_list'][i]]
        if(len(areaStart['geohash_area'])==0):
            result.append(count)
            
        else:
            while(j<len(areaStart['geohash_area'])):
                new =areaStart.reset_index()
                del new['index']
                if(new['geohash_area'][j]!= new['end'][j]):
                    count = count+1
                j=j+1
            result.append(count)
            
        i=i+1
    return result



def checkInEdge(allArea,allEnd):
    i = 0
    count = 0
    result = []
    areaLen = len(allArea['geohash_list'])
    while(i<areaLen):
        j = 0
        count = 0
        areaEnd = allEnd[allEnd.geohash_area == allArea['geohash_list'][i]]
        if(len(areaEnd)== 0):
            result.append(count)
        else:
            while(j<len(areaEnd['geohash_area'])):
                new = areaEnd.reset_index()
                del new['index']
                if(new['geohash_area'][j]!= new['start'][j]):
                    count = count+1
                j=j+1
            result.append(count)
        i=i+1
    return result

In [125]:
resultOutEdges = checkOutEdge(allArea,allStart)
resultInEdges = checkInEdge(allArea,allEnd)

In [126]:
#aggiunti attributi archi entranti ed archi uscenti
allArea['OutEdges'] = resultOutEdges
allArea['InEdges'] = resultInEdges

In [127]:
allSanFran

,driver,lat,lon,occupied,timestamp,geohash_area,distances_m,IntervalRecordTime_s,speed_km,session
0,egteir,37.75135,-122.39526,0,2008-06-04 08:25:54,9q8yyb,end,end,0.0,Morning
1,egteir,37.75143,-122.39529,0,2008-06-04 08:24:08,9q8yyb,9.281003,106.0,0.315204,Morning
2,egteir,37.75154,-122.39531,0,2008-06-04 08:20:24,9q8yyb,12.360678,224.0,0.198654,Morning
3,egteir,37.75154,-122.39531,0,2008-06-04 08:19:37,9q8yyb,0.0,47.0,0.0,Morning
4,egteir,37.75159,-122.39530,0,2008-06-04 08:18:29,9q8yyb,5.630422,68.0,0.298081,Morning
...,...,...,...,...,...,...,...,...,...,...
11219950,ashgati,37.75128,-122.39483,0,2008-05-17 13:17:48,9q8yyb,98.898318,61.0,5.836622,Afternoon
11219951,ashgati,37.75127,-122.39441,0,2008-05-17 13:16:11,9q8yyb,36.95322,97.0,1.37146,Afternoon
11219952,ashgati,37.75154,-122.39453,0,2008-05-17 13:15:06,9q8yyb,31.831416,65.0,1.762971,Afternoon
11219953,ashgati,37.75157,-122.39452,0,2008-05-17 13:14:05,9q8yyb,3.450736,61.0,0.20365,Afternoon


In [53]:
allArea.to_csv('hope.csv')

In [128]:
#Funzione che mi calcola il numero di punti per slot temporale
def coutHour(allArea,allSanFran,slot):
    i = 0
    result = []
    allAreaLen = len(allArea['geohash_list'])
    allSanFranLen = len(allSanFran['session'])
    while(i<allAreaLen):
        j=0
        area = allSanFran[allSanFran.geohash_area == allArea['geohash_list'][i]]
        area1= area[area.session == slot]
        result.append(len(area1['geohash_area']))
        i=i+1
    return result

In [129]:
resultMorning = coutHour(allArea,allSanFran,'Morning')
resultAfternoon = coutHour(allArea,allSanFran,'Afternoon')
resultEvening = coutHour(allArea,allSanFran,'Evening')
resultNight = coutHour(allArea,allSanFran,'Night')

In [130]:
allArea['VehiclesNight(24-5 am)'] = resultNight
allArea['VehiclesMorning(6-12 am)'] = resultMorning
allArea['VehiclesAfternoon(13-18 pm)'] = resultAfternoon
allArea['VehiclesEvening(19-23 pm)'] =resultEvening

In [131]:
allArea.to_csv('sanFrancisco.csv')

In [132]:
allStart.to_csv('./allStart.csv')

In [133]:
allEnd.to_csv('./allEnd.csv')

In [134]:
allSanFran.to_csv('./all1.csv')

In [139]:
#copia effettuata
allArea

,geohash_list,StartingVehicles,EndingVehicles,TotNumberRecordVehicles,maxSpeed_km/h,AverageSpeed,OutEdges,InEdges,VehiclesNight(24-5 am),VehiclesMorning(6-12 am),VehiclesAfternoon(13-18 pm),VehiclesEvening(19-23 pm),Density_n/km^2
0,9q8yuj,3,1,2293,147.771689,17.120311,3,1,1031,187,497,578,3081.160978
1,9q8yut,49,51,17256,149.291544,18.472934,37,39,4817,3471,5459,3509,23187.315238
2,9q8yuh,1,1,2193,149.594547,15.923362,1,1,669,275,641,608,2946.788498
3,9q8yum,13,17,8386,146.337848,18.008263,8,12,2277,1319,3148,1642,11268.476216
4,9q8yur,0,0,0,NaN,NaN,0,0,0,0,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,9q8yu0,0,0,152,88.868263,44.898685,0,0,33,31,48,40,204.246170
257,9q8yu9,0,3,2923,146.434622,23.417754,0,3,669,599,1064,591,3927.707605
258,9q8yu8,2,2,3734,146.365580,21.777342,2,2,898,747,1171,918,5017.468422
259,9q8yu3,2,3,2252,148.993832,16.018517,2,3,572,567,667,446,3026.068261


In [3]:
allArea = pd.read_csv('sanFrancisco.csv')
allStart = pd.read_csv('allStart.csv')
allEnd = pd.read_csv('allEnd.csv')
allSanFran = pd.read_csv('all1.csv') 

In [4]:
del allArea['Unnamed: 0']
del allStart['Unnamed: 0']
del allEnd['Unnamed: 0']
del allSanFran['Unnamed: 0']

In [5]:
#funzione che mi calcola la densità di punti per area( Area = 1.22*0.61)
def Density(allArea):
    i = 0 
    result = []
    areaLen = len(allArea['TotNumberRecordVehicles'])
    while(i<areaLen):
        count = (allArea['TotNumberRecordVehicles'][i])/(1.22*0.61)
        result.append(count)
        i=i+1
    return result

In [6]:
resultDensity = Density(allArea) #densità punti per area

In [7]:
allArea['Density_n/km^2'] = resultDensity #aggiorno il dataframe con l'attributo denistà

In [8]:
allSanFran['speed_m/s'] = allSanFran['speed_km']/3.6

In [9]:
#cambiare il tipo di oggetto della classe record time
def changetype(allBeijing):
    
    i = 0
    result = []
    allBeijingLen  = len(allBeijing['IntervalRecordTime_s'])
    while(i<allBeijingLen):
        if(allBeijing['IntervalRecordTime_s'][i]=='sosta' or allBeijing['IntervalRecordTime_s'][i]=='end'):
            result.append(0.0)
        else:
            result.append(float(allBeijing['IntervalRecordTime_s'][i]))
        i=i+1
    return result

In [10]:
resultRecordTime = changetype(allSanFran) #cambio il tipo della colonna

In [11]:
allSanFran['recordTime_float'] = resultRecordTime #aggiorno dataframe SanFrancisco

In [12]:
allSanFran['acceleration_m/s^2'] = allSanFran['speed_m/s'] / allSanFran['recordTime_float']

In [13]:
def AccMax(allBeijing,allArea):
    allAreaLen= len(allArea['geohash_list'])
    i=0
    result = []
    while(i<allAreaLen):
        area= allBeijing[allBeijing.geohash_area == allArea['geohash_list'][i]]
        acceleration = area['acceleration_m/s^2']
        result.append(acceleration.max())
        i=i+1
    return result

In [14]:
resultAcceleration = AccMax(allSanFran,allArea)

           driver       lat        lon  occupied            timestamp  \
17732      edjiye  37.77988 -122.50856         0  2008-06-06 19:37:23   
17733      edjiye  37.77774 -122.51298         0  2008-06-06 19:36:19   
17734      edjiye  37.77748 -122.51274         1  2008-06-06 19:36:12   
22890      edjiye  37.77934 -122.51154         0  2008-06-02 05:31:46   
22891      edjiye  37.77866 -122.51379         0  2008-06-02 05:30:46   
...           ...       ...        ...       ...                  ...   
11106166  ashgati  37.77996 -122.50984         1  2008-05-24 21:40:52   
11106167  ashgati  37.77994 -122.50985         1  2008-05-24 21:40:25   
11108514  ashgati  37.77876 -122.51376         0  2008-05-23 00:41:52   
11108515  ashgati  37.77875 -122.51376         1  2008-05-23 00:41:49   
11108516  ashgati  37.77872 -122.51378         1  2008-05-23 00:39:45   

         geohash_area         distances_m IntervalRecordTime_s   speed_km  \
17732          9q8yuj    607.174234846322     

Empty DataFrame
Columns: [driver, lat, lon, occupied, timestamp, geohash_area, distances_m, IntervalRecordTime_s, speed_km, session, speed_m/s, recordTime_float, acceleration_m/s^2]
Index: []
           driver       lat        lon  occupied            timestamp  \
1638       egteir  37.78765 -122.48591         0  2008-06-02 22:55:23   
1639       egteir  37.78765 -122.48591         1  2008-06-02 22:54:53   
1640       egteir  37.78764 -122.48590         1  2008-06-02 22:53:45   
1641       egteir  37.78764 -122.48581         1  2008-06-02 22:51:51   
2180       egteir  37.78800 -122.48323         1  2008-06-02 05:00:00   
...           ...       ...        ...       ...                  ...   
11035967  udwiull  37.79172 -122.47935         1  2008-06-06 07:40:21   
11035968  udwiull  37.79172 -122.47934         1  2008-06-06 07:39:00   
11035969  udwiull  37.79201 -122.48006         1  2008-06-06 07:38:00   
11048900  udwiull  37.78807 -122.48311         1  2008-05-25 21:59:48   
11071

           driver       lat        lon  occupied            timestamp  \
4094       egteir  37.78526 -122.49999         0  2008-05-31 16:58:10   
4095       egteir  37.78525 -122.49998         1  2008-05-31 16:57:47   
4096       egteir  37.78525 -122.49999         1  2008-05-31 16:56:56   
4097       egteir  37.78398 -122.49882         1  2008-05-31 16:56:05   
35468      edjiye  37.78530 -122.49985         1  2008-05-22 21:23:55   
...           ...       ...        ...       ...                  ...   
11061835  amknywr  37.78214 -122.50372         1  2008-06-05 00:51:18   
11061836  amknywr  37.78217 -122.50370         0  2008-06-05 00:50:52   
11072963  amknywr  37.78229 -122.50368         1  2008-05-23 23:39:58   
11072964  amknywr  37.78219 -122.50367         0  2008-05-23 23:39:44   
11072965  amknywr  37.78219 -122.50366         0  2008-05-23 23:38:52   

         geohash_area         distances_m IntervalRecordTime_s   speed_km  \
4094           9q8yuq   564.3608387600789     

           driver       lat        lon  occupied            timestamp  \
743        egteir  37.79719 -122.43447         0  2008-06-03 19:10:09   
744        egteir  37.79694 -122.43645         0  2008-06-03 19:09:18   
745        egteir  37.79654 -122.43931         0  2008-06-03 19:08:09   
746        egteir  37.79657 -122.43933         0  2008-06-03 19:07:09   
747        egteir  37.79642 -122.44067         0  2008-06-03 19:06:09   
...           ...       ...        ...       ...                  ...   
11114267  ashgati  37.79301 -122.43300         0  2008-05-17 14:39:24   
11114268  ashgati  37.79302 -122.43299         0  2008-05-17 14:38:24   
11114269  ashgati  37.79407 -122.43319         0  2008-05-17 14:37:23   
11114270  ashgati  37.79563 -122.43355         0  2008-05-17 14:36:48   
11114271  ashgati  37.79723 -122.43556         0  2008-05-17 14:35:47   

         geohash_area         distances_m IntervalRecordTime_s   speed_km  \
743            9q8zjb   266.1658739259663     

           driver       lat        lon  occupied            timestamp  \
12589      egteir  37.80540 -122.43448         0  2008-05-20 00:48:59   
12590      egteir  37.80599 -122.44043         0  2008-05-20 00:47:58   
12602      egteir  37.80590 -122.44167         1  2008-05-20 00:37:10   
12603      egteir  37.80631 -122.43868         1  2008-05-20 00:36:08   
12604      egteir  37.80574 -122.43487         1  2008-05-20 00:35:13   
...           ...       ...        ...       ...                  ...   
11103708  ashgati  37.80538 -122.43185         1  2008-05-27 01:12:30   
11103709  ashgati  37.80496 -122.43195         1  2008-05-27 01:11:35   
11103972  ashgati  37.80562 -122.44200         1  2008-05-26 20:32:53   
11111415  ashgati  37.80515 -122.44224         1  2008-05-19 23:27:26   
11111416  ashgati  37.80518 -122.44223         0  2008-05-19 23:26:59   

         geohash_area         distances_m IntervalRecordTime_s   speed_km  \
12589          9q8zjf  291.02416776917295     

...

           driver       lat        lon  occupied            timestamp  \
42         egteir  37.76460 -122.41630         1  2008-06-04 07:42:39   
43         egteir  37.76380 -122.41961         1  2008-06-04 07:41:34   
44         egteir  37.76500 -122.41987         0  2008-06-04 07:40:58   
46         egteir  37.76180 -122.41939         0  2008-06-04 07:38:47   
59         egteir  37.76040 -122.41924         0  2008-06-04 07:26:10   
...           ...       ...        ...       ...                  ...   
11113659  ashgati  37.76539 -122.41481         1  2008-05-18 03:06:00   
11114223  ashgati  37.76086 -122.41930         0  2008-05-17 15:21:33   
11114307  ashgati  37.76199 -122.41949         0  2008-05-17 14:02:13   
11114308  ashgati  37.76523 -122.41979         0  2008-05-17 14:01:13   
11114312  ashgati  37.76316 -122.41941         0  2008-05-17 13:57:11   

         geohash_area         distances_m IntervalRecordTime_s   speed_km  \
42             9q8yy6   318.3202650144268     

            driver       lat        lon  occupied            timestamp  \
12599       egteir  37.80424 -122.45395         1  2008-05-20 00:40:12   
94862      orkbats  37.80420 -122.45430         0  2008-05-31 15:06:03   
126999    ofjeequo  37.80420 -122.45448         1  2008-06-06 17:41:48   
158749    ajsnedsi  37.80396 -122.45379         0  2008-05-28 03:10:47   
175414     ictlypt  37.80422 -122.45439         1  2008-05-31 21:32:09   
...            ...       ...        ...       ...                  ...   
10918171  ispawwye  37.80428 -122.45391         1  2008-06-04 15:49:48   
10926788  ispawwye  37.80396 -122.45377         0  2008-05-26 23:54:30   
11014202    adkavy  37.80414 -122.45458         0  2008-06-07 22:44:15   
11048911   udwiull  37.80403 -122.45609         1  2008-05-25 21:48:48   
11103960   ashgati  37.80407 -122.45527         0  2008-05-26 20:42:03   

         geohash_area         distances_m IntervalRecordTime_s   speed_km  \
12599          9q8zj6   444.263915

           driver       lat        lon  occupied            timestamp  \
13482      egteir  37.79588 -122.47968         1  2008-05-18 08:18:54   
48674      osckro  37.79297 -122.47725         0  2008-06-04 03:48:34   
48675      osckro  37.79315 -122.47720         1  2008-06-04 03:48:25   
99863     orkbats  37.79322 -122.48084         0  2008-05-26 17:05:57   
112889     icedpa  37.79723 -122.47877         0  2008-05-28 05:08:01   
...           ...       ...        ...       ...                  ...   
11033621  udwiull  37.79308 -122.47976         1  2008-06-08 09:36:21   
11035964  udwiull  37.79327 -122.48088         0  2008-06-06 07:42:12   
11035970  udwiull  37.79713 -122.47886         1  2008-06-06 07:36:59   
11048901  udwiull  37.79575 -122.47980         1  2008-05-25 21:58:54   
11071118  amknywr  37.79312 -122.47924         1  2008-05-25 11:07:46   

         geohash_area         distances_m IntervalRecordTime_s   speed_km  \
13482          9q8zhb   443.8369631403077     

           driver       lat        lon  occupied            timestamp  \
321        egteir  37.80124 -122.46024         1  2008-06-04 02:31:46   
322        egteir  37.80131 -122.46020         0  2008-06-04 02:30:58   
323        egteir  37.80159 -122.45614         0  2008-06-04 02:28:40   
326        egteir  37.80183 -122.46332         0  2008-06-04 02:24:49   
3632       egteir  37.80322 -122.45474         1  2008-06-01 00:33:48   
...           ...       ...        ...       ...                  ...   
11111412  ashgati  37.80184 -122.46070         1  2008-05-19 23:30:22   
11111422  ashgati  37.80180 -122.45703         0  2008-05-19 23:21:10   
11111423  ashgati  37.80044 -122.46085         0  2008-05-19 23:20:10   
11111430  ashgati  37.80210 -122.45741         1  2008-05-19 23:14:29   
11114099  ashgati  37.80194 -122.46319         1  2008-05-17 17:16:32   

         geohash_area         distances_m IntervalRecordTime_s   speed_km  \
321            9q8zj3  1295.7549052908905     

           driver       lat        lon  occupied            timestamp  \
3647       egteir  37.75925 -122.48991         0  2008-06-01 00:19:48   
3648       egteir  37.75921 -122.49015         0  2008-06-01 00:18:48   
3649       egteir  37.75929 -122.49192         0  2008-06-01 00:17:47   
3650       egteir  37.75928 -122.49192         1  2008-06-01 00:17:33   
19918      edjiye  37.75907 -122.49077         1  2008-06-05 00:24:36   
...           ...       ...        ...       ...                  ...   
11076766  amknywr  37.75907 -122.49535         0  2008-05-20 13:23:26   
11092583  aslagni  37.75559 -122.49534         1  2008-05-27 19:40:07   
11092584  aslagni  37.75524 -122.49564         1  2008-05-27 19:39:33   
11092594  aslagni  37.75738 -122.48982         1  2008-05-27 19:29:54   
11105471  ashgati  37.75708 -122.49537         0  2008-05-25 08:46:04   

         geohash_area         distances_m IntervalRecordTime_s   speed_km  \
3647           9q8yu9   416.5849977403882     

In [15]:
allArea['AccelarationMax'] = resultAcceleration

In [16]:
def AccAverage(allBeijing,allArea):
    allAreaLen= len(allArea['geohash_list'])
    i=0
    result = []
    while(i<allAreaLen):
        area= allBeijing[allBeijing.geohash_area == allArea['geohash_list'][i]]
        acceleration = area['acceleration_m/s^2']
        
        result.append(acceleration.mean())
        i=i+1
    return result

In [17]:
resultAverageAcc = AccAverage(allSanFran,allArea)

In [19]:
allArea['AverageAcceleration_m/s^2'] = resultAverageAcc

In [21]:
allArea

,geohash_list,StartingVehicles,EndingVehicles,TotNumberRecordVehicles,maxSpeed_km/h,AverageSpeed,OutEdges,InEdges,VehiclesNight(24-5 am),VehiclesMorning(6-12 am),VehiclesAfternoon(13-18 pm),VehiclesEvening(19-23 pm),Density_n/km^2,AccelarationMax,AverageAcceleration_m/s^2
0,9q8yuj,3,1,2293,147.771689,17.120311,3,1,1031,187,497,578,3081.160978,24.952845,0.186663
1,9q8yut,49,51,17256,149.291544,18.472934,37,39,4817,3471,5459,3509,23187.315238,40.088502,0.155163
2,9q8yuh,1,1,2193,149.594547,15.923362,1,1,669,275,641,608,2946.788498,41.408578,0.152058
3,9q8yum,13,17,8386,146.337848,18.008263,8,12,2277,1319,3148,1642,11268.476216,39.528499,0.171032
4,9q8yur,0,0,0,NaN,NaN,0,0,0,0,0,0,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,9q8yu0,0,0,152,88.868263,44.898685,0,0,33,31,48,40,204.246170,1.417922,0.212889
257,9q8yu9,0,3,2923,146.434622,23.417754,0,3,669,599,1064,591,3927.707605,13.115238,0.149117
258,9q8yu8,2,2,3734,146.365580,21.777342,2,2,898,747,1171,918,5017.468422,20.328553,0.155068
259,9q8yu3,2,3,2252,148.993832,16.018517,2,3,572,567,667,446,3026.068261,25.642442,0.220884


In [19]:
allArea.to_csv('./sanFrancisco.csv')

Inizio con i punti di interesse

In [22]:
import geopandas as gpd

museum = gpd.read_file('./interessi/museumS.geojson')
attraction = gpd.read_file('./interessi/attractionS.geojson')
university = gpd.read_file('./interessi/universityS.geojson')
bank = gpd.read_file('./interessi/bankS.geojson')
hospital = gpd.read_file('./interessi/hospitalS.geojson')
dentist = gpd.read_file('./interessi/dentistS.geojson')
cinema = gpd.read_file('./interessi/cinemaS.geojson')
stadium = gpd.read_file('./interessi/stadiumS.geojson')
fuel = gpd.read_file('./interessi/fuelS.geojson')
zoo = gpd.read_file('./interessi/zooS.geojson')
pharmacy = gpd.read_file('./interessi/pharmacyS.geojson')
busStation = gpd.read_file('./interessi/bus_stationS.geojson')
ruins = gpd.read_file('./interessi/ruinsS.geojson')
swimmingPool = gpd.read_file('./interessi/swimmingPoolS.geojson')
taxi_stop = gpd.read_file('./interessi/taxiS.geojson')
company = gpd.read_file('./interessi/company.geojson')
library = gpd.read_file('./interessi/libraryS.geojson')
theatre = gpd.read_file('./interessi/theatreS.geojson')

In [23]:
def checkInterest(allArea,interest):
    i = 0
    count = 0
    result = []
    interestLen = len(interest['geometry'])
    areaLen = len(allArea['geohash_list'])
    while(i<areaLen):
        j = 0
        count = 0
        while(j<interestLen):
            if(interest['geometry'][j].centroid.within(allArea['geometry'][i])):
                count = count+1
            
            j=j+1
        result.append(count)    
        i=i+1
    return result

In [34]:
resultMuseum = checkInterest(geo_tassellation,museum)
resultAttraction = checkInterest(geo_tassellation,attraction)
resultUniversity= checkInterest(geo_tassellation,university)
resultBank = checkInterest(geo_tassellation,bank)
resultHospital = checkInterest(geo_tassellation,hospital)
resultDentist = checkInterest(geo_tassellation,dentist)
resultCinema = checkInterest(geo_tassellation,cinema)
resultStadium = checkInterest(geo_tassellation,stadium)
resultPharmacy = checkInterest(geo_tassellation,pharmacy)
resultFuel = checkInterest(geo_tassellation,fuel)
resultZoo = checkInterest(geo_tassellation,zoo)
resultBus = checkInterest(geo_tassellation,busStation)
resultRuins = checkInterest(geo_tassellation,ruins)
resultSP = checkInterest(geo_tassellation,swimmingPool)
resultTaxi = checkInterest(geo_tassellation,taxi_stop)
resultCompany = checkInterest(geo_tassellation,company)
resultLibrary = checkInterest(geo_tassellation,library)
resultTheatre = checkInterest(geo_tassellation,theatre)

In [35]:
resultCulture= []
i = 0

while(i<len(resultAttraction)):
    count = 0
    count = resultMuseum[i]+ resultAttraction[i]+resultRuins[i]
    resultCulture.append(count)
    i=i+1

In [36]:
resultFinance= []
i =0

while(i<len(resultBank)):
    count = 0
    count = resultBank[i] + resultCompany[i]
    resultFinance.append(count)
    i=i+1

In [37]:
resultEnterteiment= []
i =0

while(i<len(resultCinema)):
    count = 0
    count = resultCinema[i] + resultTheatre[i] + resultZoo[i] + resultStadium[i] +resultSP[i]
    resultEnterteiment.append(count)
    i=i+1

In [38]:
resultTrasportation = []
i = 0
while(i<len(resultBus)):
    count = 0
    count = resultBus[i] +resultFuel[i] + resultTaxi[i]
    resultTrasportation.append(count)
    i = i+1

In [39]:
resultEducation = []
i = 0
while(i<len(resultLibrary)):
    count = 0
    count = resultUniversity[i] + resultLibrary[i]
    resultEducation.append(count)
    i=i+1

In [40]:
resultHealth= []
i =0

while(i<len(resultHospital)):
    count = 0
    count = resultHospital[i] + resultDentist[i] + resultPharmacy[i]
    resultHealth.append(count)
    i=i+1

In [41]:
#mi sono scordato il numero totale di punti di interesse

i = 0
totalInterest = []
while(i<len(resultBus)):
    count = 0
    count = resultCulture[i]+ resultFinance[i]+resultEnterteiment[i]+resultHealth[i]+resultTrasportation[i]+resultEducation[i]
    totalInterest.append(count)
    i=i+1


In [42]:
allArea['NumberTotalPointInterest'] = totalInterest
allArea['NumberCulturalPoint("Museum-Attraction-Ruins")'] = resultCulture
allArea['NumberFinancePoint("Bank-Company")'] = resultBank
allArea['NumberHealthPoint("Hospital-Dentist-Pharmacy")'] = resultHealth
allArea['NumberTrasportationPoint("Fuel-BusStation-TaxiStop")'] = resultTrasportation
allArea['NumberEducationPoint("University-Library")'] = resultEducation
allArea['NumberEnterteimentPoint("Cinema-Theatre-Zoo-Stadium-SwimmingPool")'] = resultEnterteiment

In [43]:
allArea

,geohash_list,StartingVehicles,EndingVehicles,TotNumberRecordVehicles,maxSpeed_km/h,AverageSpeed,OutEdges,InEdges,VehiclesNight(24-5 am),VehiclesMorning(6-12 am),...,Density_n/km^2,AccelarationMax,AverageAcceleration_m/s^2,NumberTotalPointInterest,"NumberCulturalPoint(""Museum-Attraction-Ruins"")","NumberFinancePoint(""Bank-Company"")","NumberHealthPoint(""Hospital-Dentist-Pharmacy"")","NumberTrasportationPoint(""Fuel-BusStation-TaxiStop"")","NumberEducationPoint(""University-Library"")","NumberEnterteimentPoint(""Cinema-Theatre-Zoo-Stadium-SwimmingPool"")"
0,9q8yuj,3,1,2293,147.771689,17.120311,3,1,1031,187,...,3081.160978,24.952845,0.186663,0,0,0,0,0,0,0
1,9q8yut,49,51,17256,149.291544,18.472934,37,39,4817,3471,...,23187.315238,40.088502,0.155163,2,2,0,0,0,0,0
2,9q8yuh,1,1,2193,149.594547,15.923362,1,1,669,275,...,2946.788498,41.408578,0.152058,4,2,0,1,1,0,0
3,9q8yum,13,17,8386,146.337848,18.008263,8,12,2277,1319,...,11268.476216,39.528499,0.171032,3,0,0,2,1,0,0
4,9q8yur,0,0,0,NaN,NaN,0,0,0,0,...,0.000000,NaN,NaN,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,9q8yu0,0,0,152,88.868263,44.898685,0,0,33,31,...,204.246170,1.417922,0.212889,1,0,0,0,1,0,0
257,9q8yu9,0,3,2923,146.434622,23.417754,0,3,669,599,...,3927.707605,13.115238,0.149117,2,0,0,1,0,1,0
258,9q8yu8,2,2,3734,146.365580,21.777342,2,2,898,747,...,5017.468422,20.328553,0.155068,0,0,0,0,0,0,0
259,9q8yu3,2,3,2252,148.993832,16.018517,2,3,572,567,...,3026.068261,25.642442,0.220884,0,0,0,0,0,0,0


In [44]:
allArea.to_csv('sanFrancisco.csv')

Verificare il numero di punti durante la settimana

In [45]:
allSanFran['timestamp']=pd.to_datetime(allSanFran['timestamp'],format='%Y-%m-%d %H:%M:%S')

In [46]:
#utilizzo la funzione weekday che mi permette di assegnare un numero da 0 a 6 per indicare il giorno della settimana
i = 0
result = []
l = len(allSanFran['timestamp'])
while(i<l):
    day = allSanFran['timestamp'][i].weekday()
    result.append(day)
    i = i+1

In [47]:
allSanFran['weekday'] = result

In [54]:
allSanFran.to_csv('./all1.csv')

In [49]:
#funzione che mi calcola il numero di punti per area nel weekend
def checkWeek(allArea,allSanFran):
    i = 0
    result = []
    lenArea = len(allArea['geohash_list'])
    while(i<lenArea):
        area= allSanFran[allSanFran.geohash_area == allArea['geohash_list'][i]]
        area1 = area[area.weekday == 5]
        area2 = area[area.weekday == 6]
        result.append(len(area1)+len(area2))    
        i=i+1
    return result

In [50]:
resultWeek = checkWeek(allArea,allSanFran)

In [51]:
allArea['RecordWeekend'] = resultWeek

In [52]:
def checkWeekday(allArea,allSanFran):
    i = 0
    result = []
    lenArea = len(allArea['geohash_list'])
    while(i<lenArea):
        area= allSanFran[allSanFran.geohash_area == allArea['geohash_list'][i]]
        area1 = area[area.weekday == 0]
        area2 = area[area.weekday == 1]
        area3 = area[area.weekday == 2]
        area4 = area[area.weekday == 3]
        area5 = area[area.weekday == 4]
        result.append(len(area1)+len(area2)+len(area3)+len(area4)+len(area5))    
        i=i+1
    return result

In [53]:
resultWeekday = checkWeekday(allArea,allSanFran)

In [54]:
allArea['RecordWeek'] = resultWeekday

In [55]:
allArea

,geohash_list,StartingVehicles,EndingVehicles,TotNumberRecordVehicles,maxSpeed_km/h,AverageSpeed,OutEdges,InEdges,VehiclesNight(24-5 am),VehiclesMorning(6-12 am),...,AverageAcceleration_m/s^2,NumberTotalPointInterest,"NumberCulturalPoint(""Museum-Attraction-Ruins"")","NumberFinancePoint(""Bank-Company"")","NumberHealthPoint(""Hospital-Dentist-Pharmacy"")","NumberTrasportationPoint(""Fuel-BusStation-TaxiStop"")","NumberEducationPoint(""University-Library"")","NumberEnterteimentPoint(""Cinema-Theatre-Zoo-Stadium-SwimmingPool"")",RecordWeekend,RecordWeek
0,9q8yuj,3,1,2293,147.771689,17.120311,3,1,1031,187,...,0.186663,0,0,0,0,0,0,0,851,1442
1,9q8yut,49,51,17256,149.291544,18.472934,37,39,4817,3471,...,0.155163,2,2,0,0,0,0,0,6636,10620
2,9q8yuh,1,1,2193,149.594547,15.923362,1,1,669,275,...,0.152058,4,2,0,1,1,0,0,931,1262
3,9q8yum,13,17,8386,146.337848,18.008263,8,12,2277,1319,...,0.171032,3,0,0,2,1,0,0,2766,5620
4,9q8yur,0,0,0,NaN,NaN,0,0,0,0,...,NaN,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,9q8yu0,0,0,152,88.868263,44.898685,0,0,33,31,...,0.212889,1,0,0,0,1,0,0,52,100
257,9q8yu9,0,3,2923,146.434622,23.417754,0,3,669,599,...,0.149117,2,0,0,1,0,1,0,1041,1882
258,9q8yu8,2,2,3734,146.365580,21.777342,2,2,898,747,...,0.155068,0,0,0,0,0,0,0,1359,2375
259,9q8yu3,2,3,2252,148.993832,16.018517,2,3,572,567,...,0.220884,0,0,0,0,0,0,0,776,1476


In [56]:
allArea.to_csv('./sanFrancisco.csv')

In [58]:
allArea['NumberTotalPointInterest'].sum()

1034

In [2]:
AllArea = pd.read_csv('sanFrancisco.csv')

In [5]:
AllArea.head()

,geohash_list,StartingVehicles,EndingVehicles,TotNumberRecordVehicles,maxSpeed_km/h,AverageSpeed,OutEdges,InEdges,VehiclesNight(24-5 am),VehiclesMorning(6-12 am),...,AverageAcceleration_m/s^2,NumberTotalPointInterest,"NumberCulturalPoint(""Museum-Attraction-Ruins"")","NumberFinancePoint(""Bank-Company"")","NumberHealthPoint(""Hospital-Dentist-Pharmacy"")","NumberTrasportationPoint(""Fuel-BusStation-TaxiStop"")","NumberEducationPoint(""University-Library"")","NumberEnterteimentPoint(""Cinema-Theatre-Zoo-Stadium-SwimmingPool"")",RecordWeekend,RecordWeek
0,9q8yuj,3,1,2293,147.771689,17.120311,3,1,1031,187,...,0.186663,0,0,0,0,0,0,0,851,1442
1,9q8yut,49,51,17256,149.291544,18.472934,37,39,4817,3471,...,0.155163,2,2,0,0,0,0,0,6636,10620
2,9q8yuh,1,1,2193,149.594547,15.923362,1,1,669,275,...,0.152058,4,2,0,1,1,0,0,931,1262
3,9q8yum,13,17,8386,146.337848,18.008263,8,12,2277,1319,...,0.171032,3,0,0,2,1,0,0,2766,5620
4,9q8yur,0,0,0,NaN,NaN,0,0,0,0,...,NaN,0,0,0,0,0,0,0,0,0


In [4]:
del AllArea['Unnamed: 0']

In [7]:
for c in AllArea.columns:
    print (c)

geohash_list
StartingVehicles
EndingVehicles
TotNumberRecordVehicles
maxSpeed_km/h
AverageSpeed
OutEdges
InEdges
VehiclesNight(24-5 am)
VehiclesMorning(6-12 am)
VehiclesAfternoon(13-18 pm)
VehiclesEvening(19-23 pm)
Density_n/km^2
AccelarationMax
AverageAcceleration_m/s^2
NumberTotalPointInterest
NumberCulturalPoint("Museum-Attraction-Ruins")
NumberFinancePoint("Bank-Company")
NumberHealthPoint("Hospital-Dentist-Pharmacy")
NumberTrasportationPoint("Fuel-BusStation-TaxiStop")
NumberEducationPoint("University-Library")
NumberEnterteimentPoint("Cinema-Theatre-Zoo-Stadium-SwimmingPool")
RecordWeekend
RecordWeek


In [1]:
import pandas as pd

In [2]:
allArea = pd.read_csv('sanFrancisco.csv')
del allArea['Unnamed: 0']

In [7]:
allArea

,geohash_list,StartingVehicles,EndingVehicles,TotNumberRecordVehicles,maxSpeed_km/h,AverageSpeed,OutEdges,InEdges,VehiclesNight(24-5 am),VehiclesMorning(6-12 am),...,AverageAcceleration_m/s^2,NumberTotalPointInterest,"NumberCulturalPoint(""Museum-Attraction-Ruins"")","NumberFinancePoint(""Bank-Company"")","NumberHealthPoint(""Hospital-Dentist-Pharmacy"")","NumberTrasportationPoint(""Fuel-BusStation-TaxiStop"")","NumberEducationPoint(""University-Library"")","NumberEnterteimentPoint(""Cinema-Theatre-Zoo-Stadium-SwimmingPool"")",RecordWeekend,RecordWeek
0,9q8yuj,3,1,2293,147.771689,17.120311,3,1,1031,187,...,0.186663,0,0,0,0,0,0,0,851,1442
1,9q8yut,49,51,17256,149.291544,18.472934,37,39,4817,3471,...,0.155163,2,2,0,0,0,0,0,6636,10620
2,9q8yuh,1,1,2193,149.594547,15.923362,1,1,669,275,...,0.152058,4,2,0,1,1,0,0,931,1262
3,9q8yum,13,17,8386,146.337848,18.008263,8,12,2277,1319,...,0.171032,3,0,0,2,1,0,0,2766,5620
4,9q8yur,0,0,0,NaN,NaN,0,0,0,0,...,NaN,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,9q8yu0,0,0,152,88.868263,44.898685,0,0,33,31,...,0.212889,1,0,0,0,1,0,0,52,100
257,9q8yu9,0,3,2923,146.434622,23.417754,0,3,669,599,...,0.149117,2,0,0,1,0,1,0,1041,1882
258,9q8yu8,2,2,3734,146.365580,21.777342,2,2,898,747,...,0.155068,0,0,0,0,0,0,0,1359,2375
259,9q8yu3,2,3,2252,148.993832,16.018517,2,3,572,567,...,0.220884,0,0,0,0,0,0,0,776,1476


In [9]:
allArea.head(50)

,geohash_list,StartingVehicles,EndingVehicles,TotNumberRecordVehicles,maxSpeed_km/h,AverageSpeed,OutEdges,InEdges,VehiclesNight(24-5 am),VehiclesMorning(6-12 am),...,AverageAcceleration_m/s^2,NumberTotalPointInterest,"NumberCulturalPoint(""Museum-Attraction-Ruins"")","NumberFinancePoint(""Bank-Company"")","NumberHealthPoint(""Hospital-Dentist-Pharmacy"")","NumberTrasportationPoint(""Fuel-BusStation-TaxiStop"")","NumberEducationPoint(""University-Library"")","NumberEnterteimentPoint(""Cinema-Theatre-Zoo-Stadium-SwimmingPool"")",RecordWeekend,RecordWeek
0,9q8yuj,3,1,2293,147.771689,17.120311,3,1,1031,187,...,0.186663,0,0,0,0,0,0,0,851,1442
1,9q8yut,49,51,17256,149.291544,18.472934,37,39,4817,3471,...,0.155163,2,2,0,0,0,0,0,6636,10620
2,9q8yuh,1,1,2193,149.594547,15.923362,1,1,669,275,...,0.152058,4,2,0,1,1,0,0,931,1262
3,9q8yum,13,17,8386,146.337848,18.008263,8,12,2277,1319,...,0.171032,3,0,0,2,1,0,0,2766,5620
4,9q8yur,0,0,0,NaN,NaN,0,0,0,0,...,NaN,0,0,0,0,0,0,0,0,0
5,9q8yuz,0,0,1494,148.327955,15.175686,0,0,638,276,...,0.152451,2,0,0,1,0,1,0,501,993
6,9q8yuw,4,2,2945,147.268768,13.213262,4,2,741,392,...,0.166454,5,5,0,0,0,0,0,885,2060
7,9q8yun,0,0,13,26.647334,7.631524,0,0,0,0,...,0.034964,1,1,0,0,0,0,0,8,5
8,9q8yux,0,0,102,96.392548,14.987547,0,0,35,15,...,0.489791,5,4,0,1,0,0,0,40,62
9,9q8yuq,0,0,2052,131.498735,8.340321,0,0,188,16,...,0.181797,0,0,0,0,0,0,0,769,1283
